In [ ]:
pip install scikit-fuzzy

     |████████████████████████████████| 993 kB 10.0 MB/s 
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-py3-none-any.whl size=894089 sha256=d1fbcd32d70c3fe1840749cf115663ffd243a814276d8a56c16d820cd4936297
  Stored in directory: /root/.cache/pip/wheels/d5/74/fc/38588a3d2e3f34f74588e6daa3aa5b0a322bd6f9420a707131
Successfully built scikit-fuzzy


In [ ]:
import numpy as np
import skfuzzy as fuzz
import skfuzzy.control as ctrl

#一些有關的function

> https://pythonhosted.org/scikit-fuzzy/api/skfuzzy.control.html#skfuzzy.control.Antecedent





In [ ]:
#x 參數範圍
star_range = np.arange(1,5) 
price_range = np.arange(0,500)
distance_range = np.arange(0,5000)
#y 參數範圍
recommend_range = np.arange(1,10)

# 定義fuzzy set和其函數
# Antecedent(universe, label)[source]
# Antecedent (input/sensor) variable for a fuzzy control system.
# universe : array-like(Universe variable. Must be 1-dimensional and convertible to a NumPy array.)
# label : string(Name of the universe variable.)
star = ctrl.Antecedent(star_range,'star')
price=ctrl.Antecedent(price_range,'price')
distance=ctrl.Antecedent(distance_range,'distance')
recommend=ctrl.Consequent(recommend_range,'recommend')


#三角形->trimp，梯形->trapmf
star['L']=fuzz.trimf(star_range,[1,1,2]) 
star['M']=fuzz.trimf(star_range,[2,3,4])
star['H']=fuzz.trapmf(star_range,[3,4,5,5])

price['L']=fuzz.trapmf(price_range,[0,0,60,250]) #60元到250算便宜
price['M']=fuzz.trimf(price_range,[250,400,700])
price['H']=fuzz.trapmf(price_range,[700,1500,3000,3000])

distance['L']=fuzz.trapmf(distance_range,[0,0,200,300]) #200公尺到300公尺內算近
distance['M']=fuzz.trimf(distance_range,[300,500,1000])
distance['H']=fuzz.trapmf(distance_range,[1000,3000,5000,5000])

recommend['vL']=fuzz.trimf(recommend_range,[1,1,2])
recommend['L']=fuzz.trimf(recommend_range,[2,3,4]) #推薦指數1~10
recommend['M']=fuzz.trimf(recommend_range,[4,5,6])
recommend['H']=fuzz.trimf(recommend_range,[6,7,8])
recommend['vH']=fuzz.trapmf(recommend_range,[8,8,9,10])


In [ ]:
#設定defuzzify方法
recommend.defuzzify_method='centroid'

In [ ]:
'''
rule1 = ctrl.Rule(antecedent=((star['L'] & price['H']) |
                (price['H'] & distance['H']) |
                (star['L'] & distance['H'])),
        consequent=recommend['vL'], label='rule 1')
'''
#Rule1:[星數低]且[價格高]且[距離遠]->推薦程度[極低]
rule1 = ctrl.Rule(antecedent=((star['L'] & price['H'] & distance['H']) ),
        consequent=recommend['vL'], label='rule 1')

#Rule2:[星數低]且[價格中]且[距離遠]->推薦程度[低]
rule2 = ctrl.Rule(antecedent=((star['L'] & price['M'] & distance['H']) ),
        consequent=recommend['L'], label='rule 2')

#Rule3:[星數中]且[價格中]且[距離中]->推薦程度[中]
rule3 = ctrl.Rule(antecedent=((star['M'] & price['M'] & distance['M']) ),
        consequent=recommend['M'], label='rule 3')          

#Rule4:[星數高]且[價格低]且[距離中]->推薦程度[高]
rule4 = ctrl.Rule(antecedent=((star['H'] & price['L'] & distance['M']) ),
        consequent=recommend['H'], label='rule 4')  
          
#Rule5:[星數高]且[價格低]且[距離近]->推薦程度[極高]
rule5 = ctrl.Rule(antecedent=((star['H'] & price['L'] & distance['L'])),
        consequent=recommend['vH'], label='rule 5')           

In [ ]:
#系統和環境初始化
system = ctrl.ControlSystem(rules=[rule1,rule2,rule3,rule4,rule5])
sim = ctrl.ControlSystemSimulation(system)

In [ ]:
sim.input['star']=5
sim.input['price']=1
sim.input['distance']=1
sim.compute()
output = sim.output['recommend']
print(output)

8.222222222222223
